### Step 0: Load the libraries

In [7]:
# Data manipulation
import pandas as pd
import numpy as np
# loading data from sql
import sqlite3
# time tracking
from tqdm import tqdm

### Step 1: Load the data

In [79]:
# Relative path to the SQLite database file
database_file = "./data/corpus.sqlite3"

# Connect to the database
conn = sqlite3.connect(database_file)

# Load data into a DataFrame
df_posts = pd.read_sql_query("SELECT * FROM Posts", conn)

# Close the connection
conn.close()


### Step 2: Filter and preprocess

In [80]:
df_posts.head()

df_posts['Headline'] = df_posts['Headline'].fillna('').str.strip()
df_posts['Body'] = df_posts['Body'].fillna('').str.strip()
df_posts['FullText'] = df_posts['Headline'] + ' ' + df_posts['Body']
df_posts['FullText'] = df_posts['FullText'].str.strip()
df = df_posts[{'ID_Post', 'FullText'}]
df.head().style

,ID_Post,FullText
0,1,
1,2,"Newsletter ""DER STANDARD"" Ich bin begeistert von den STANDARD - Newslettern. In keiner anderen E-Zeitung gibt es eine solche Viel= falt und so interessante Kommentare bzw. eine so genaue Berichterstattung. Macht weiter so, ich finde es jedenfalls toll. Johann Radakovits"
2,3,"Auch begeistert... ... Aber momentan funktioniert das Abmelden oder Adressen ändern überhaupt nicht! In USA macht man dies auf den Servern einfacher. Ansonsten ist DER STANDARD ja wirklich das Beste, was es in Österreich gibt... Weiter im ""Text"" ;-)carpeDiem(-;"
3,4,"Abmeldeprobleme Es ist ganz einfach nervend! Vor kurzem habe ich den news-letter angemeldet in der hoffnungsvollen Gewissheit (weil vertrauenserweckend angekündigt), ihn jederzeit wieder stornieren zu können. Aber leider führt kein Weg dahin. Ich wäre dankbar für eine diesbezügliche Hilfe des Lesers dieser eMail."
4,5,und sie als mitarbeiter sind natuerlich objektiv


### Step 3: Load and test the pretrained model

In [81]:
#!pip install germansentiment
from germansentiment import SentimentModel
sentiment_model = SentimentModel()


print( sentiment_model.predict_sentiment(["Mit einem guten Ergebniss"], output_probabilities =True ))
print( sentiment_model.predict_sentiment(["Du bist hässlich und dumm"], output_probabilities = True ))
print( sentiment_model.predict_sentiment(["Sie fährt ein grünes Auto."], output_probabilities = True ))


print("Positivity score:", sentiment_model.predict_sentiment(["Mit einem guten Ergebniss"], 
                                         output_probabilities =True )[1][0][0][1])

print("Negativty score:", sentiment_model.predict_sentiment(["Mit einem guten Ergebniss"], 
                                         output_probabilities =True )[1][0][1][1])

print("Neutral score:", sentiment_model.predict_sentiment(["Mit einem guten Ergebniss"], 
                                         output_probabilities =True )[1][0][2][1])

(['positive'], [[['positive', 0.9939589500427246], ['negative', 0.005358279217034578], ['neutral', 0.0006827253382652998]]])
(['negative'], [[['positive', 0.001920510665513575], ['negative', 0.9980425834655762], ['neutral', 3.68434630217962e-05]]])
(['neutral'], [[['positive', 2.1235788153717294e-05], ['negative', 5.393160245148465e-05], ['neutral', 0.9999247789382935]]])
Positivity score: 0.9939589500427246
Negativty score: 0.005358279217034578
Neutral score: 0.0006827253382652998


In [82]:
# Initialize sentiment_score column
#df["sentiment_score"] = -999.999 

<ipython-input-82-c7c96146e7ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment_score"] = -999.999


### Step 4: Generate the sentiment feature by applying the model on the df

In [83]:
def calculate_scentiment_score(text):
    sentiment_result = sentiment_model.predict_sentiment([text], output_probabilities=True)
    # Extract probabilities
    positive_prob = sentiment_result[1][0][0][1]  # Probability of 'positive'
    negative_prob = sentiment_result[1][0][1][1]  # Probability of 'negative'
    neutral_prob = sentiment_result[1][0][2][1]  # Probability of 'neutral'

    # Determine sentiment score based on probabilities
    if sentiment_result[0][0]  == 'positive':
        sentiment_score = positive_prob
    elif sentiment_result[0][0]  == 'negative':
        sentiment_score = (-1) * negative_prob
    elif sentiment_result[0][0] == 'neutral':
        sentiment_score = 0

    return sentiment_score

calculate_scentiment_score("Mit einem guten Ergebnis")

0.9937969446182251

In [ ]:

def generate_sentiment_feature():
    """Integrate all features into the dataframe based on the full text column."""
    # Use tqdm to create a progress bar for iterating through the DataFrame
    tqdm.pandas(desc="Calculating Sentiment Scores")
    
    # Apply the sentiment prediction with progress tracking
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        
        if row['sentiment_score'] == -999.999:

            # Update the sentiment_score for the current record
            df.at[index, 'sentiment_score'] = calculate_scentiment_score(row['FullText'])
            
        else: 
            continue

# Apply feature extraction
generate_sentiment_feature()


Processing rows:  99%|███████████████████████████████████████████████████████▋| 1005306/1011773 [20:27:46<12:47,  8.42it/s]

In [104]:
df.head().style

,ID_Post,FullText,sentiment_score
0,1,,-0.872363
1,2,"Newsletter ""DER STANDARD"" Ich bin begeistert von den STANDARD - Newslettern. In keiner anderen E-Zeitung gibt es eine solche Viel= falt und so interessante Kommentare bzw. eine so genaue Berichterstattung. Macht weiter so, ich finde es jedenfalls toll. Johann Radakovits",0.999520
2,3,"Auch begeistert... ... Aber momentan funktioniert das Abmelden oder Adressen ändern überhaupt nicht! In USA macht man dies auf den Servern einfacher. Ansonsten ist DER STANDARD ja wirklich das Beste, was es in Österreich gibt... Weiter im ""Text"" ;-)carpeDiem(-;",0.944021
3,4,"Abmeldeprobleme Es ist ganz einfach nervend! Vor kurzem habe ich den news-letter angemeldet in der hoffnungsvollen Gewissheit (weil vertrauenserweckend angekündigt), ihn jederzeit wieder stornieren zu können. Aber leider führt kein Weg dahin. Ich wäre dankbar für eine diesbezügliche Hilfe des Lesers dieser eMail.",-0.997048
4,5,und sie als mitarbeiter sind natuerlich objektiv,0.000000


save the results

In [105]:
df.to_pickle("./data/sentiment_dataframe.pkl")

In [107]:
df = pd.read_pickle("./data/sentiment_dataframe.pkl")

In [106]:
df

,ID_Post,FullText,sentiment_score
0,1,,-0.872363
1,2,"Newsletter ""DER STANDARD"" Ich bin begeistert v...",0.999520
2,3,Auch begeistert... ... Aber momentan funktioni...,0.944021
3,4,Abmeldeprobleme Es ist ganz einfach nervend!\r...,-0.997048
4,5,und sie als mitarbeiter sind natuerlich objektiv,0.000000
...,...,...,...
1011768,1011769,zwischen der beendigung eines arbeitsverhältni...,-999.999000
1011769,1011770,du sagst es ja im namen selbst: wegen eindicku...,-999.999000
1011770,1011771,Was genau haben Sie denn nicht verstanden? Ich...,-999.999000
1011771,1011772,irgendwie widersprechen Sie sich in Ihrem Post...,-999.999000
